In [1]:
# Install required libraries
!pip install textstat -q

# Import libraries
print("Importing Libraries")

import os
import glob
import numpy as np
import pandas as pd
from tqdm import tqdm
from collections import Counter
import nltk
from nltk.tokenize import word_tokenize, sent_tokenize
from nltk.sentiment.vader import SentimentIntensityAnalyzer
import textstat
from transformers import AutoTokenizer, AutoModel, GPT2Tokenizer, GPT2LMHeadModel
import torch
import lightgbm as lgb
from catboost import CatBoostClassifier
import optuna
from sklearn.metrics import accuracy_score
from sklearn.decomposition import PCA
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics.pairwise import cosine_similarity
from scipy.stats import entropy

import warnings
warnings.filterwarnings('ignore')
print("Libraries Imported Successfully")

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 175.3/175.3 kB 11.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 939.4/939.4 kB 44.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 69.1 MB/s eta 0:00:00
Importing Libraries


2025-07-13 21:18:36.759210: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1752441516.960485      19 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1752441517.016525      19 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


Libraries Imported Successfully


In [2]:
# Download required NLTK data
nltk.download('punkt', quiet=True)
nltk.download('vader_lexicon', quiet=True)

# Configuration
class Config:
    BASE_PATH = "/kaggle/input/fake-or-real-the-impostor-hunt/data/"
    TRAIN_PATH = os.path.join(BASE_PATH, "train")
    TEST_PATH = os.path.join(BASE_PATH, "test")
    TRAIN_CSV = os.path.join(BASE_PATH, "train.csv")
    SUBMISSION_FILE = "submission.csv"
    N_SPLITS = 5
    RANDOM_STATE = 42
    MODEL_NAME = "allenai/scibert_scivocab_uncased"
    MAX_LENGTH = 512
    BATCH_SIZE = 16
    N_TRIALS = 100

config = Config()

# Load RoBERTa tokenizer and model
print("Loading RoBERTa")
tokenizer = AutoTokenizer.from_pretrained(config.MODEL_NAME)
model = AutoModel.from_pretrained(config.MODEL_NAME)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)
model.eval()
print("Loaded successfully")

# Load GPT-2 for perplexity
print("Loading GPT-2")
gpt2_tokenizer = GPT2Tokenizer.from_pretrained("gpt2-medium")
gpt2_model = GPT2LMHeadModel.from_pretrained("gpt2-medium").to(device)
gpt2_model.eval()
print("GPT-2 loaded successfully")

# Data loading
def read_text_files_robust(df, path):
    texts_1, texts_2 = [], []
    missing_files = []
    all_dirs = glob.glob(os.path.join(path, 'article_*'))
    dir_map = {int(os.path.basename(p).replace('article_', '')): p for p in all_dirs}
    for _, row in tqdm(df.iterrows(), total=len(df), desc=f"Reading files from {os.path.basename(path)}"):
        article_id = row['id']
        dir_path = dir_map.get(article_id)
        if dir_path:
            try:
                with open(os.path.join(dir_path, 'file_1.txt'), 'r', encoding='utf-8') as f:
                    text = f.read().strip() or "No content available"
                    texts_1.append(text)
                    if not text:
                        missing_files.append((article_id, 'file_1.txt'))
            except FileNotFoundError:
                texts_1.append("No content available")
                missing_files.append((article_id, 'file_1.txt'))
            try:
                with open(os.path.join(dir_path, 'file_2.txt'), 'r', encoding='utf-8') as f:
                    text = f.read().strip() or "No content available"
                    texts_2.append(text)
                    if not text:
                        missing_files.append((article_id, 'file_2.txt'))
            except FileNotFoundError:
                texts_2.append("No content available")
                missing_files.append((article_id, 'file_2.txt'))
        else:
            texts_1.append("No content available")
            texts_2.append("No content available")
            missing_files.append((article_id, 'both'))
    df['text_1'] = texts_1
    df['text_2'] = texts_2
    print(f"\nMissing or empty files in {os.path.basename(path)}: {len(missing_files)}")
    if missing_files:
        print("Missing/empty file details (article_id, file):", missing_files[:5], "..." if len(missing_files) > 5 else "")
    return df

def load_data(config):
    train_df = pd.read_csv(config.TRAIN_CSV)
    test_dirs = glob.glob(os.path.join(config.TEST_PATH, 'article_*'))
    if not test_dirs:
        raise FileNotFoundError(f"No 'article_*' directories found in {config.TEST_PATH}")
    test_ids = [int(os.path.basename(p).replace('article_', '')) for p in test_dirs]
    test_df = pd.DataFrame(sorted(test_ids), columns=['id'])
    print("\nTrain set size (before augmentation):", len(train_df))
    print("Test set size:", len(test_df))
    print("Train label distribution:", train_df['real_text_id'].value_counts(normalize=True).to_dict())
    train_df = read_text_files_robust(train_df, config.TRAIN_PATH)
    test_df = read_text_files_robust(test_df, config.TEST_PATH)
    # Data augmentation
    train_df = augment_data(train_df)
    print("Train set size (after augmentation):", len(train_df))
    print("Augmented train label distribution:", train_df['real_text_id'].value_counts(normalize=True).to_dict())
    return train_df, test_df

def augment_data(df):
    swapped_df = df.copy()
    swapped_df['text_1'], swapped_df['text_2'] = df['text_2'], df['text_1']
    swapped_df['real_text_id'] = swapped_df['real_text_id'].apply(lambda x: 2 if x == 1 else 1)
    return pd.concat([df, swapped_df], ignore_index=True)

# Feature extraction
def get_text_features(text):
    if not isinstance(text, str) or not text.strip():
        return {
            'char_count': 0,
            'word_count': 0,
            'flesch_reading_ease': 0,
            'punctuation_count': 0,
            'ttr': 0
        }
    words = word_tokenize(text.lower())
    sentences = sent_tokenize(text)
    
    char_count = len(text)
    word_count = len(words)
    flesch_reading_ease = textstat.flesch_reading_ease(text)
    punctuation_count = sum(1 for char in text if char in '.,!?;')
    unique_words = len(set(words))
    ttr = unique_words / (word_count + 1e-9)
    
    return {
        'char_count': char_count,
        'word_count': word_count,
        'flesch_reading_ease': flesch_reading_ease,
        'punctuation_count': punctuation_count,
        'ttr': ttr
    }

def get_watermark_features(text):
    if not isinstance(text, str) or not text.strip():
        return {'emdash_count': 0}
    emdash_count = text.count('\u2014')
    return {'emdash_count': emdash_count}

def get_perplexity(text, tokenizer, model, max_length=512):
    if not isinstance(text, str) or not text.strip():
        return 0.0
    inputs = tokenizer(text, return_tensors="pt", truncation=True, max_length=max_length).to(device)
    with torch.no_grad():
        outputs = model(**inputs, labels=inputs["input_ids"])
        loss = outputs.loss.item()
    perplexity = np.exp(loss)
    return min(perplexity, 1000.0)  # Clip at 1000

def get_sentence_perplexity(text, tokenizer, model, max_length=512):
    if not isinstance(text, str) or not text.strip():
        return 0.0, 0.0
    sentences = sent_tokenize(text)
    if not sentences:
        return 0.0, 0.0
    perplexities = []
    for sent in sentences:
        inputs = tokenizer(sent, return_tensors="pt", truncation=True, max_length=max_length).to(device)
        with torch.no_grad():
            outputs = model(**inputs, labels=inputs["input_ids"])
            loss = outputs.loss.item()
        perplexities.append(min(np.exp(loss), 1000.0))  # Clip at 1000
    return np.mean(perplexities), np.std(perplexities)

def get_burstiness(text):
    if not isinstance(text, str) or not text.strip():
        return 0.0
    sentences = sent_tokenize(text)
    if len(sentences) < 2:
        return 0.0
    sentence_lengths = [len(word_tokenize(sent)) for sent in sentences]
    mean_length = np.mean(sentence_lengths)
    std_length = np.std(sentence_lengths)
    return std_length / (mean_length + 1e-9)

def get_roberta_embeddings(texts, tokenizer, model, config):
    embeddings = []
    for i in tqdm(range(0, len(texts), config.BATCH_SIZE), desc="Generating RoBERTa embeddings"):
        batch_texts = texts[i:i + config.BATCH_SIZE]
        inputs = tokenizer(
            batch_texts, padding=True, truncation=True, max_length=config.MAX_LENGTH,
            return_tensors="pt"
        ).to(device)
        with torch.no_grad():
            outputs = model(**inputs)
        batch_embeddings = outputs.last_hidden_state[:, 0, :].cpu().numpy()
        embeddings.append(batch_embeddings)
    return np.vstack(embeddings)

def create_features(df, tokenizer, model, gpt2_tokenizer, gpt2_model, config):
    short_jaccard_pairs = []
    short_kl_pairs = []
    
    # Basic text features
    features_1 = df['text_1'].apply(get_text_features).apply(pd.Series)
    features_2 = df['text_2'].apply(get_text_features).apply(pd.Series)
    feature_cols = list(features_1.columns)
    print("\nText feature stats (train set):" if 'real_text_id' in df.columns else "\nText feature stats (test set):")
    for col in feature_cols:
        print(f"{col} (text_1):", features_1[col].describe().to_dict())
        print(f"{col} (text_2):", features_2[col].describe().to_dict())

    # Watermark features
    watermark_1 = df['text_1'].apply(get_watermark_features).apply(pd.Series)
    watermark_2 = df['text_2'].apply(get_watermark_features).apply(pd.Series)
    watermark_cols = list(watermark_1.columns)
    print("\nWatermark feature stats:")
    for col in watermark_cols:
        print(f"{col} (text_1):", watermark_1[col].describe().to_dict())
        print(f"{col} (text_2):", watermark_2[col].describe().to_dict())

    # Perplexity features
    df['perplexity_1'] = df['text_1'].apply(lambda x: get_perplexity(x, gpt2_tokenizer, gpt2_model))
    df['perplexity_2'] = df['text_2'].apply(lambda x: get_perplexity(x, gpt2_tokenizer, gpt2_model))
    df['perplexity_diff'] = df['perplexity_1'] - df['perplexity_2']
    print("\nPerplexity stats:")
    print("perplexity_1:", df['perplexity_1'].describe().to_dict())
    print("perplexity_2:", df['perplexity_2'].describe().to_dict())
    print("perplexity_diff:", df['perplexity_diff'].describe().to_dict())

    # Burstiness features
    df['burstiness_1'] = df['text_1'].apply(get_burstiness)
    df['burstiness_2'] = df['text_2'].apply(get_burstiness)
    df['burstiness_diff'] = df['burstiness_1'] - df['burstiness_2']
    print("\nBurstiness stats:")
    print("burstiness_1:", df['burstiness_1'].describe().to_dict())
    print("burstiness_2:", df['burstiness_2'].describe().to_dict())
    print("burstiness_diff:", df['burstiness_diff'].describe().to_dict())

    # RoBERTa embeddings
    texts_1 = df['text_1'].tolist()
    texts_2 = df['text_2'].tolist()
    emb_1 = get_roberta_embeddings(texts_1, tokenizer, model, config)
    emb_2 = get_roberta_embeddings(texts_2, tokenizer, model, config)
    print("\nEmbedding stats (first dimension):")
    print("emb_1[0]:", pd.Series(emb_1[:, 0]).describe().to_dict())
    print("emb_2[0]:", pd.Series(emb_2[:, 0]).describe().to_dict())

    # Cosine similarity
    cosine_sim = [cosine_similarity(emb_1[i:i+1], emb_2[i:i+1])[0][0] for i in range(len(emb_1))]
    df['cosine_similarity'] = cosine_sim
    print("\nCosine similarity stats:", df['cosine_similarity'].describe().to_dict())

    # Embedding differences with PCA
    emb_diff = emb_1 - emb_2
    if 'real_text_id' in df.columns:
        pca = PCA(n_components=0.95, random_state=config.RANDOM_STATE)
        emb_diff_pca = pca.fit_transform(emb_diff)
        config.pca = pca
    else:
        emb_diff_pca = config.pca.transform(emb_diff)
    emb_cols = [f'emb_diff_pca_{i}' for i in range(emb_diff_pca.shape[1])]
    df[emb_cols] = pd.DataFrame(emb_diff_pca, index=df.index)
    print("\nEmbedding difference PCA stats (first 3 components):")
    for i in range(min(3, len(emb_cols))):
        print(f"{emb_cols[i]}:", df[emb_cols[i]].describe().to_dict())

    # Sentiment polarity
    sid = SentimentIntensityAnalyzer()
    df['polarity_1'] = df['text_1'].apply(lambda x: sid.polarity_scores(x)['compound'] if isinstance(x, str) and x.strip() else 0)
    df['polarity_2'] = df['text_2'].apply(lambda x: sid.polarity_scores(x)['compound'] if isinstance(x, str) and x.strip() else 0)
    print("\nSentiment polarity stats:")
    print("polarity_1:", df['polarity_1'].describe().to_dict())
    print("polarity_2:", df['polarity_2'].describe().to_dict())

    # Jaccard similarity with unigram fallback
    def jaccard_similarity(text1, text2):
        if not isinstance(text1, str) or not text1.strip() or not isinstance(text2, str) or not text2.strip():
            return 0.0
        words1 = word_tokenize(text1.lower())
        words2 = word_tokenize(text2.lower())
        if len(words1) < 2 or len(words2) < 2:
            short_jaccard_pairs.append((text1[:50], text2[:50]))
            # Character-level Jaccard
            set1 = set(text1.lower())
            set2 = set(text2.lower())
            intersection = len(set1.intersection(set2))
            union = len(set1.union(set2))
            return intersection / union if union > 0 else 0.0
        set1, set2 = set(nltk.bigrams(words1)), set(nltk.bigrams(words2))
        intersection = len(set1.intersection(set2))
        union = len(set1.union(set2))
        return intersection / union if union > 0 else 0.0
    df['bigram_jaccard'] = df.apply(lambda x: jaccard_similarity(x['text_1'], x['text_2']), axis=1)
    print("\nBigram Jaccard stats:", df['bigram_jaccard'].describe().to_dict())
    print(f"Number of pairs using unigram Jaccard (<2 words): {len(short_jaccard_pairs)}")
    if short_jaccard_pairs:
        print("Sample short Jaccard pairs (text_1, text_2):", short_jaccard_pairs[:5], "..." if len(short_jaccard_pairs) > 5 else "")

    # N-gram KL Divergence with unigram fallback
    def get_ngram_kl_divergence(text1, text2, n=2):
        if not isinstance(text1, str) or not text1.strip() or not isinstance(text2, str) or not text2.strip():
            return 0.0
        words1 = word_tokenize(text1.lower())
        words2 = word_tokenize(text2.lower())
        if len(words1) < n or len(words2) < n:
            short_kl_pairs.append((text1[:50], text2[:50]))
            ngrams1 = Counter(words1)  # Unigram fallback
            ngrams2 = Counter(words2)
        else:
            ngrams1 = Counter(nltk.ngrams(words1, n))
            ngrams2 = Counter(nltk.ngrams(words2, n))
        if not ngrams1 or not ngrams2:
            return 0.0
        vocab = set(ngrams1.keys()) | set(ngrams2.keys())
        p = np.array([ngrams1.get(ngram, 1e-9) / sum(ngrams1.values()) for ngram in vocab])
        q = np.array([ngrams2.get(ngram, 1e-9) / sum(ngrams2.values()) for ngram in vocab])
        return entropy(p, q)
    df['ngram_kl_divergence'] = df.apply(lambda x: get_ngram_kl_divergence(x['text_1'], x['text_2']), axis=1)
    print("\nN-gram KL divergence stats:", df['ngram_kl_divergence'].describe().to_dict())
    print(f"Number of pairs with short texts for KL divergence (<2 words): {len(short_kl_pairs)}")
    if short_kl_pairs:
        print("Sample short KL pairs (text_1, text_2):", short_kl_pairs[:5], "..." if len(short_kl_pairs) > 5 else "")

    # Difference features
    for col in feature_cols:
        df[f'{col}_diff'] = features_1[col].astype(float) - features_2[col].astype(float)
        print(f"\n{col}_diff stats:", df[f'{col}_diff'].describe().to_dict())

    # Text length diagnostics
    print("\nText length stats (train set):" if 'real_text_id' in df.columns else "\nText length stats (test set):")
    text_1_lengths = df['text_1'].apply(lambda x: len(word_tokenize(x)) if isinstance(x, str) else 0)
    text_2_lengths = df['text_2'].apply(lambda x: len(word_tokenize(x)) if isinstance(x, str) else 0)
    print("Text_1 word count stats:", text_1_lengths.describe().to_dict())
    print("Text_2 word count stats:", text_2_lengths.describe().to_dict())
    short_pairs = df[df[['text_1', 'text_2']].apply(lambda x: len(word_tokenize(x['text_1'])) < 2 or len(word_tokenize(x['text_2'])) < 2, axis=1)]
    print(f"Number of pairs with short texts (<2 words): {len(short_pairs)}")
    if len(short_pairs) > 0:
        print("Sample short pairs (id, text_1, text_2):", [(row['id'], row['text_1'][:50], row['text_2'][:50]) for _, row in short_pairs.head(5).iterrows()], "..." if len(short_pairs) > 5 else "")

    # All features before selection
    all_feature_cols = emb_cols + [f'{col}_diff' for col in feature_cols] + ['cosine_similarity', 'bigram_jaccard', 'ngram_kl_divergence', 'perplexity_diff', 'burstiness_diff']

    # Select top PCA components based on importance > 10
    if 'real_text_id' in df.columns:
        X = df[all_feature_cols]
        y = df['real_text_id'].apply(lambda x: 0 if x == 1 else 1)
        cat_model = CatBoostClassifier(loss_function='Logloss', random_seed=config.RANDOM_STATE, verbose=0)
        cat_model.fit(X, y)
        feature_importance = pd.DataFrame({
            'feature': all_feature_cols,
            'importance': cat_model.get_feature_importance()
        })
        print("\nPreliminary feature importance (CatBoost):")
        print(feature_importance.sort_values('importance', ascending=False).head(10).to_string(index=False))
        # Filter PCA components with importance > 10
        config.top_emb_cols = feature_importance[feature_importance['feature'].str.startswith('emb_diff_pca_') & (feature_importance['importance'] > 10)]['feature'].tolist()
        print(f"\nSelected PCA components (importance > 10): {config.top_emb_cols}")
    else:
        config.top_emb_cols = getattr(config, 'top_emb_cols', emb_cols)

    final_feature_cols = config.top_emb_cols + [f'{col}_diff' for col in feature_cols if col not in ['avg_sentence_length']] + ['cosine_similarity', 'bigram_jaccard', 'ngram_kl_divergence', 'perplexity_diff', 'burstiness_diff']
    return df, final_feature_cols

# Training and prediction with Optuna
def train_and_predict(train_df, test_df, feature_cols, config):
    X = train_df[feature_cols]
    y = train_df['real_text_id'].apply(lambda x: 0 if x == 1 else 1)
    X_test = test_df[feature_cols]

    def objective(trial):
        params = {
            'loss_function': 'Logloss',
            'iterations': trial.suggest_int('iterations', 100, 800),
            'learning_rate': trial.suggest_float('learning_rate', 0.001, 0.3, log=True),
            'depth': trial.suggest_int('depth', 2, 4),
            'colsample_bylevel': trial.suggest_float('colsample_bylevel', 0.3, 1.0),
            'subsample': trial.suggest_float('subsample', 0.3, 1.0),
            'min_child_samples': trial.suggest_float('min_child_samples', 2, 15),
            'l2_leaf_reg': trial.suggest_float('l2_leaf_reg', 20, 100.0, log=True),
            'border_count': trial.suggest_int('border_count', 32, 254),
            'random_seed': config.RANDOM_STATE,
            'verbose': 0
        }

        skf = StratifiedKFold(n_splits=config.N_SPLITS, shuffle=True, random_state=config.RANDOM_STATE)
        oof_preds = np.zeros(len(X))
        fold_accuracies = []
        for fold, (train_idx, val_idx) in enumerate(skf.split(X, y)):
            X_train, y_train = X.iloc[train_idx], y.iloc[train_idx]
            X_val, y_val = X.iloc[val_idx], y.iloc[val_idx]
            model = CatBoostClassifier(**params)
            model.fit(
                X_train, y_train,
                eval_set=[(X_val, y_val)],
                early_stopping_rounds=100,
                verbose=False
            )
            val_preds_proba = model.predict_proba(X_val)[:, 1]
            oof_preds[val_idx] = val_preds_proba
            val_preds = np.round(val_preds_proba)
            fold_accuracy = accuracy_score(y_val, val_preds)
            fold_accuracies.append(fold_accuracy)
            print(f"\nFold {fold+1} validation accuracy: {fold_accuracy:.5f}")
            print(f"Fold {fold+1} prediction probability stats:", pd.Series(val_preds_proba).describe().to_dict())
        mean_accuracy = np.mean(fold_accuracies)
        std_accuracy = np.std(fold_accuracies)
        print(f"\nMean fold accuracy: {mean_accuracy:.5f}, Std: {std_accuracy:.5f}")
        return mean_accuracy

    study = optuna.create_study(direction='maximize', sampler=optuna.samplers.TPESampler(seed=config.RANDOM_STATE))
    print("\nStarting Optuna optimization")
    study.optimize(objective, n_trials=config.N_TRIALS)
    best_params = study.best_params
    print("\nBest Optuna trial value (mean CV accuracy):", study.best_value)
    print("Best hyperparameters:", best_params)

    skf = StratifiedKFold(n_splits=config.N_SPLITS, shuffle=True, random_state=config.RANDOM_STATE)
    oof_preds = np.zeros(len(train_df))
    test_preds = np.zeros(len(test_df))
    feature_importances = []
    fold_accuracies = []

    for fold, (train_idx, val_idx) in enumerate(skf.split(X, y)):
        print(f"\n===== Fold {fold+1}/{config.N_SPLITS} =====")
        X_train, y_train = X.iloc[train_idx], y.iloc[train_idx]
        X_val, y_val = X.iloc[val_idx], y.iloc[val_idx]
        print(f"Training samples: {len(train_idx)}, Validation samples: {len(val_idx)}")
        print(f"Training label distribution:", pd.Series(y_train).value_counts(normalize=True).to_dict())

        model = CatBoostClassifier(**best_params)
        model.fit(
            X_train, y_train,
            eval_set=[(X_val, y_val)],
            early_stopping_rounds=100,
            verbose=False
        )

        val_preds_proba = model.predict_proba(X_val)[:, 1]
        oof_preds[val_idx] = val_preds_proba
        val_preds = np.round(val_preds_proba)
        fold_accuracy = accuracy_score(y_val, val_preds)
        fold_accuracies.append(fold_accuracy)
        print(f"Fold {fold+1} validation accuracy: {fold_accuracy:.5f}")
        print(f"Fold {fold+1} prediction probability stats:", pd.Series(val_preds_proba).describe().to_dict())
        test_pred_stats = pd.Series(model.predict_proba(X_test)[:, 1]).describe().to_dict()
        print(f"Fold {fold+1} test prediction probability stats:", test_pred_stats)
        test_preds += model.predict_proba(X_test)[:, 1] / config.N_SPLITS

        fold_importance = pd.DataFrame({
            'feature': feature_cols,
            'importance': model.get_feature_importance()
        })
        feature_importances.append(fold_importance)

    feature_importance_df = pd.concat(feature_importances).groupby('feature').mean().reset_index()
    feature_importance_df = feature_importance_df.sort_values('importance', ascending=False)
    print("\nFull Feature Importance:")
    print(feature_importance_df.to_string(index=False))

    oof_accuracy = accuracy_score(y, np.round(oof_preds))
    print(f"\nOverall CV Accuracy: {oof_accuracy:.5f}")
    print(f"Mean fold accuracy: {np.mean(fold_accuracies):.5f}, Std: {np.std(fold_accuracies):.5f}")
    print(f"OOF prediction probability stats:", pd.Series(oof_preds).describe().to_dict())
    print(f"Test prediction probability stats:", pd.Series(test_preds).describe().to_dict())
    print(f"Best hyperparameters:", best_params)
    return test_preds

# Load data
print("Loading data")
train_df, test_df = load_data(config)

# Create features
print("Creating features")
train_df, feature_cols = create_features(train_df, tokenizer, model, gpt2_tokenizer, gpt2_model, config)
test_df, _ = create_features(test_df, tokenizer, model, gpt2_tokenizer, gpt2_model, config)

# Train and predict
print("Training initiated")
test_predictions_proba = train_and_predict(train_df, test_df, feature_cols, config)

# Generate submission
print("Making predictions")
final_predictions_class = (test_predictions_proba > 0.5).astype(int)
submission_df = pd.DataFrame({'id': test_df['id'], 'real_text_id': final_predictions_class + 1})
submission_df.to_csv(config.SUBMISSION_FILE, index=False)
print("Complete.")

Loading RoBERTa


config.json:   0%|          | 0.00/385 [00:00<?, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

pytorch_model.bin:   0%|          | 0.00/442M [00:00<?, ?B/s]

Loaded successfully
Loading GPT-2


...


Fold 5 validation accuracy: 0.94737
Fold 5 prediction probability stats: {'count': 38.0, 'mean': 0.48672956135963014, 'std': 0.42963588036091566, 'min': 0.025213486776648107, '25%': 0.04093247085485044, '50%': 0.4407876864900715, '75%': 0.9634363694290556, 'max': 0.9719186041538477}

Mean fold accuracy: 0.93158, Std: 0.05669

Fold 1 validation accuracy: 0.92105
Fold 1 prediction probability stats: {'count': 38.0, 'mean': 0.46814468552613747, 'std': 0.4075648145287267, 'min': 0.03580595137170925, '25%': 0.06582840430363125, '50%': 0.3632206916826844, '75%': 0.9551412671714792, 'max': 0.9745638821277106}

Fold 2 validation accuracy: 0.92105
Fold 2 prediction probability stats: {'count': 38.0, 'mean': 0.5190501916991895, 'std': 0.4225921583204441, 'min': 0.012395885394503455, '25%': 0.040966531969093264, '50%': 0.4493964422484794, '75%': 0.9632989108657855, 'max': 0.9872722657357101}

Fold 3 validation accuracy: 0.97368
Fold 3 prediction probability stats: {'count': 38.0, 'mean': 0.54922

[I 2025-07-13 21:25:30,810] Trial 98 finished with value: 0.9421052631578949 and parameters: {'iterations': 748, 'learning_rate': 0.23833386885929567, 'depth': 4, 'colsample_bylevel': 0.4511429574350977, 'subsample': 0.7664896465705346, 'min_child_samples': 6.550144647273323, 'l2_leaf_reg': 21.75149685310235, 'border_count': 156}. Best is trial 4 with value: 0.9526315789473685.



Fold 4 validation accuracy: 0.97368
Fold 4 prediction probability stats: {'count': 38.0, 'mean': 0.48064810861107254, 'std': 0.4715244262654327, 'min': 0.003676793051222593, '25%': 0.007445440388039393, '50%': 0.25412192202371603, '75%': 0.9818420372134606, 'max': 0.9965581933409379}

Fold 5 validation accuracy: 0.92105
Fold 5 prediction probability stats: {'count': 38.0, 'mean': 0.47385536807659373, 'std': 0.4362560310701407, 'min': 0.011296951424311665, '25%': 0.03211752762561635, '50%': 0.3100961008283658, '75%': 0.9753274355946615, 'max': 0.9866506948070178}

Mean fold accuracy: 0.94211, Std: 0.02578

Fold 1 validation accuracy: 0.86842
Fold 1 prediction probability stats: {'count': 38.0, 'mean': 0.46184972791331147, 'std': 0.40730985702317696, 'min': 0.03613137255975858, '25%': 0.06320061959309259, '50%': 0.41324086032739527, '75%': 0.9442214073641094, 'max': 0.9717748308523463}

Fold 2 validation accuracy: 0.92105
Fold 2 prediction probability stats: {'count': 38.0, 'mean': 0.54

[I 2025-07-13 21:25:32,265] Trial 99 finished with value: 0.9368421052631579 and parameters: {'iterations': 691, 'learning_rate': 0.12728285657333793, 'depth': 4, 'colsample_bylevel': 0.852586097693191, 'subsample': 0.3521067382003625, 'min_child_samples': 5.30295948377623, 'l2_leaf_reg': 46.866196679985386, 'border_count': 110}. Best is trial 4 with value: 0.9526315789473685.



Fold 5 validation accuracy: 0.94737
Fold 5 prediction probability stats: {'count': 38.0, 'mean': 0.4842738233200496, 'std': 0.4797380000437397, 'min': 0.0015865619256534283, '25%': 0.0035272582197966233, '50%': 0.3648827635714562, '75%': 0.996440309419643, 'max': 0.9978231548935086}

Mean fold accuracy: 0.93684, Std: 0.03939

Best Optuna trial value (mean CV accuracy): 0.9526315789473685
Best hyperparameters: {'iterations': 145, 'learning_rate': 0.22413234378101138, 'depth': 4, 'colsample_bylevel': 0.8658781436815228, 'subsample': 0.5132296384213595, 'min_child_samples': 3.2697374820829905, 'l2_leaf_reg': 60.157302950529136, 'border_count': 130}

===== Fold 1/5 =====
Training samples: 152, Validation samples: 38
Training label distribution: {0: 0.5, 1: 0.5}
Fold 1 validation accuracy: 0.92105
Fold 1 prediction probability stats: {'count': 38.0, 'mean': 0.4662610222448332, 'std': 0.4009369739588508, 'min': 0.049674414995084126, '25%': 0.08330180009691082, '50%': 0.30838441828246493, '7